In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from konlpy.tag import Kkma, Komoran, Okt, Hannanum
from datetime import datetime

In [2]:
cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient-basic-windows.x64-21.9.0.0.0dbru\instantclient_21_9")

In [4]:
#db에 연결
conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
#커서 생성
cs = conn.cursor()

In [5]:
#sql 실행
sql = "select * from CBASIC_DATA where DATA_TITLE is not null"
rs = cs.execute(sql)

In [6]:
#행,열 추출
row = cs.fetchall()
colname = cs.description
col = []
for i in colname:
    col.append(i[0])

In [7]:
ctdf =  pd.DataFrame(row,columns = col)

In [8]:
ctdf

,CODE_URL,DATA_TITLE,DATA_COUNT,DATA_LINK,EXTRACT_DATE,DATE_WRITE
0,02,저도 mr4 화이트 도착했네요,376,/bbs/qb_free/views/8166441?page=39,20230516,05-02
1,02,에어컨도 특가 이벤트 있나요,290,/bbs/qb_free/views/8166422?page=39,20230516,05-02
2,02,새 배너 나왔네요,179,/bbs/qb_free/views/8166418?page=39,20230516,05-02
3,02,MR4가 도착했네요,291,/bbs/qb_free/views/8166413?page=39,20230516,05-02
4,02,아사히슈퍼드라이 생맥주 국내와 해외버전 차이점,382,/bbs/qb_free/views/8166401?page=39,20230516,05-02
...,...,...,...,...,...,...
983,02,누구에게나 그럴듯한 계획은있다,291,/bbs/qb_free/views/8165317?page=41,20230516,05-01
984,02,이런건 휴지에 싸서 버려주세요,1.4k,/bbs/qb_free/views/8165313?page=41,20230516,05-01
985,02,하늘이시여,150,/bbs/qb_free/views/8165312?page=41,20230516,05-01
986,02,월페이퍼엔진중에 마음에 드는 바탕화면을 찾았습니다,454,/bbs/qb_free/views/8165309?page=41,20230516,05-01


In [9]:
#konlpy 세팅
okt = Okt()
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()

In [10]:
title=np.array(ctdf['DATA_TITLE'])

In [13]:
#제목에서 명사만 추출해서 리스트로(2차원 리스트)
sep_title=[]
for i in range(len(title)):
    sep_title.append(okt.nouns(title[i]))

In [14]:
#count 정수로 변환
count = list(ctdf['DATA_COUNT'])
count1=[]
count2=[]
for i in count:
    temp = i.replace("k","000")
    count1.append(temp)

for j in count1:
    t = j.replace(".","")
    count2.append(t)

count_int = list(map(int,count2))

In [15]:
#날짜 데이터 문자열->DATE 변환
date=list(ctdf['DATE_WRITE'])
Date=[]
for i in range(len(date)):
    Date.append(datetime.strptime(date[i],'%m-%d'))

In [16]:
#명사 데이터프레임 생성
Ndf = pd.DataFrame(columns = ['word','count','date'])
for i in range(len(sep_title)):
    for j in range(len(sep_title[i])):
        row = pd.DataFrame([[sep_title[i][j], count_int[i], Date[i]]], columns = ['word','count','date'])
        Ndf = pd.concat([Ndf,row])

In [18]:
Ndf

,word,count,date
0,저,376,1900-05-02
0,화이트,376,1900-05-02
0,도착,376,1900-05-02
0,에어컨,290,1900-05-02
0,특,290,1900-05-02
...,...,...,...
0,마음,454,1900-05-01
0,바탕화면,454,1900-05-01
0,출근,380,1900-05-01
0,안,380,1900-05-01
